In [35]:
%matplotlib inline
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
import math

To-do List:
1. Make sure that predictions are made for every game in the dataset
2. Make sure that the GameNum is the index value for the created y_pred set
3. Add variable importances to the model output for our presentation 

In [36]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('final.csv')
all_games = pd.read_csv('allgames_future_v6.csv')

Making a test set with only two teams

In [37]:
#df.set_index('GameNum', inplace = True)

In [38]:
# drop unneccessary columns
df.drop(['Unnamed: 0','Date', 'Time', 'W/L',
                      'PF', 'PA', 'SPEC', 'Week',
                      'SPEC_opposing', 'Team_opposing'],axis=1, inplace = True)

In [39]:

df.shape

(6814, 437)

In [40]:
# what is the overall variance in the attributes
attributes = ['PASS', 'PBLK', 'RECV', 'RUN', 'RBLK', 'RDEF', 'TACK', 'PRSH', 'COV']
# for attribute in attributes:
#     print(df[attribute].std())

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

# Cross Validation for Gradient Boosting - Sklearn

In [42]:
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor

## Predict EVERYTHING

In [43]:
grouped = df.groupby('Team')

In [44]:
gbr_predictions = DataFrame()
bagr_predictions = DataFrame()
target = DataFrame()

In [45]:
best_boost = DataFrame(columns = ['Attribute', 'Team', 'loss', 'learning_rate', 'max_depth', 'n_estimators'])
best_bag = DataFrame(columns = ['Attribute', 'Team', 'max_samples', 'n_estimators'])

In [46]:
import pprint
import operator
import itertools

# Option 1 -- Fast AF

In [47]:
gbr_parameters = {'loss': ['ls','huber'],
                  'learning_rate': [0.001,0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6],
                  'max_depth': [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)],
                  'n_estimators': [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)]}

bag_parameters = {'max_samples': [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)],
                  'n_estimators': [int(x) for x in np.linspace(start = 2, stop = 20, num = 10)]}

In [48]:
%%time
pp = pprint.PrettyPrinter(indent=4)
for attribute in attributes:
    # Lists to hold on to the generated values
    gbr_preds = []
    bagr_preds = []
    t = []
    teams = []
    games = []
    
    columns = []
    
    if attribute == 'RBLK':
        for i in range(1, 17):
            columns.append('RBLK_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RUN_lag' + str(i))
        columns.append('TACK_avg3_opposing')
        columns.append('RDEF_avg3_opposing')
        
    elif attribute == 'PBLK':
        for i in range(1, 17):
            columns.append('PBLK_lag' + str(i))
            columns.append('PASS_lag' + str(i))
        columns.append('PRSH_avg3_opposing')
        columns.append('COV_avg3_opposing')
    
    elif attribute == 'RECV':
        for i in range(1, 17):
            columns.append('PBLK_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            columns.append('PASS_lag' + str(i))
        columns.append('TACK_avg3_opposing')
        columns.append('PRSH_avg3_opposing')
        columns.append('COV_avg3_opposing')
    
    elif attribute == 'PASS':
        for i in range(1, 17):
            columns.append('PBLK_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RUN_lag' + str(i))
        columns.append('PRSH_avg3_opposing')
        columns.append('COV_avg3_opposing')
    
    elif attribute == 'RUN':
        for i in range(1, 17):
            columns.append('RBLK_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RUN_lag' + str(i))
        columns.append('TACK_avg3_opposing')
        columns.append('RDEF_avg3_opposing')
        
    elif attribute == 'TACK':
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            columns.append('COV_lag' + str(i))
        columns.append('RBLK_avg3_opposing')
        columns.append('RUN_avg3_opposing')
    
    elif attribute == 'RDEF':
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
        columns.append('RBLK_avg3_opposing')
        columns.append('RUN_avg3_opposing')
    
    elif attribute == 'PRSH':
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('PRSH_lag' + str(i))
            columns.append('COV_lag' + str(i))
        columns.append('PBLK_avg3_opposing')
        columns.append('PASS_avg3_opposing')
    
    elif attribute == 'COV':
        for i in range(1, 17):
            columns.append('PRSH_lag' + str(i))
            columns.append('COV_lag' + str(i))
        columns.append('PBLK_avg3_opposing')
        columns.append('RECV_avg3_opposing')
        columns.append('PASS_avg3_opposing')
    
    for team in grouped:
        data = team[1]
        
        
        # Train test split
        train = data[data['Season'] < 2017]
        test = data[data['Season'] >= 2017]
        
        y_train = train[attribute]
        X_train = train[columns]
        
        y_test = test[attribute]
        X_test = test[columns]
        
        team = test['Team'].values
        gameNum = test['GameNum'].values
        
        
        # Gradient Boosting Regressor
        gbr = GradientBoostingRegressor()
        
        # Bagging Regressor
        bag = BaggingRegressor()
        
        # Train models
        gbr.fit(X_train, y_train)
        bag.fit(X_train, y_train)
        
        
        
        # Now Cross Validate
        grid_search = GridSearchCV(gbr, gbr_parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        
        print(attribute, "GBR", grid_search.best_params_)

        gbr_regressor = GradientBoostingRegressor(
            loss=grid_search.best_params_['loss'],
            max_depth=grid_search.best_params_['max_depth'],
            n_estimators=grid_search.best_params_['n_estimators'],
            learning_rate=grid_search.best_params_['learning_rate']
        )
        
        #Save best model parameters
        best_boost = best_boost.append({'Attribute' : attribute,
                                        'Team' : team[0],
                                        'loss' : grid_search.best_params_['loss'],
                                        'learning_rate' : grid_search.best_params_['learning_rate'],
                                        'max_depth' : grid_search.best_params_['max_depth'],
                                        'n_estimators' : grid_search.best_params_['n_estimators']},
                                      ignore_index = True)
        
        
        
        # CV BAG
        grid_search = GridSearchCV(bag, bag_parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        
        print(attribute, "BAG", grid_search.best_params_)

        bag_regressor = BaggingRegressor(
            
            max_samples=grid_search.best_params_['max_samples'],
            n_estimators=grid_search.best_params_['n_estimators']
        )
        
        # Save the best model parameters
        best_bag = best_bag.append({'Attribute' : attribute,
                                    'Team' : team[0],
                                    'max_samples' : grid_search.best_params_['max_samples'],
                                    'n_estimators' : grid_search.best_params_['n_estimators']},
                                  ignore_index = True)
        
        
        # Train cross validated models
        gbr_regressor.fit(X_train, y_train)
        
        bag_regressor.fit(X_train, y_train)
        
        
        # Make predictions
        gbr_hat = gbr_regressor.predict(X_test)
        bagr_hat = bag_regressor.predict(X_test)
        
        
        # Save each team's data
        for i in range(len(y_test)):
            t.append(y_test.iloc[i])
            gbr_preds.append(gbr_hat[i])
            bagr_preds.append(bagr_hat[i])
            teams.append(team[i])
            games.append(gameNum[i])
    
    # Save data for each attribute    
    gbr_predictions[attribute] = gbr_preds
    bagr_predictions[attribute] = bagr_preds
    target[attribute] = t
    
    gbr_predictions['Team'] = teams
    gbr_predictions['GameNum'] = games
    bagr_predictions['Team'] = teams
    bagr_predictions['GameNum'] = games
    target['Team'] = teams
    target['GameNum'] = games
    
    # Takes a while to run, so this shows progress
    print('\n\n\n\n\n\nFinished creating preditcions for:', attribute, '\n\n\n\n\n\n\n')

Fitting 5 folds for each of 550 candidates, totalling 2750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 794 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 1754 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-1)]: Done 2750 out of 2750 | elapsed:  1.0min finished
C:\Users\chris\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


('PASS', 'GBR', {'n_estimators': 10, 'loss': 'ls', 'learning_rate': 0.2, 'max_depth': 2})
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    8.3s finished
C:\Users\chris\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


('PASS', 'BAG', {'n_estimators': 16, 'max_samples': 16})
Fitting 5 folds for each of 550 candidates, totalling 2750 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 904 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 2184 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 2750 out of 2750 | elapsed:   42.9s finished
C:\Users\chris\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


('COV', 'GBR', {'n_estimators': 6, 'loss': 'huber', 'learning_rate': 0.4, 'max_depth': 2})
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 456 tasks      | elapsed:    6.5s


('COV', 'BAG', {'n_estimators': 14, 'max_samples': 2})
('\n\n\n\n\n\nFinished creating preditcions for:', 'COV', '\n\n\n\n\n\n\n')
Wall time: 4h 14min 29s


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    7.2s finished
C:\Users\chris\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [49]:
gbr_predictions.to_csv('gbr_predictions_v3.csv')
bagr_predictions.to_csv('bagr_predictions_v3.csv')

In [50]:
average_columns = [x + '_avg3' for x in attributes]
avg3 = df[df['Season'] >= 2017][average_columns]
avg3.columns = [x for x in attributes]
avg3.reset_index(inplace = True)

In [51]:
RMSEs = DataFrame(index = attributes)
RMSEs['Actual Variation'] = [target[x].std() for x in attributes]

In [52]:
import math

def calculate_RMSE(target_df, prediction_df):
    results = []
    for col in target_df.columns.values:
        target = target_df[col]
        prediction = prediction_df[col]
        RMSE = math.sqrt(((prediction - target) ** 2).mean())
        
        results.append(RMSE)
    
    return results

In [53]:
# Calculate RMSEs for 3 different models: GBR, Bagging, average of last 3 games
RMSEs['GBR'] = calculate_RMSE(target[attributes], gbr_predictions[attributes])
RMSEs['Bagging'] = calculate_RMSE(target[attributes], bagr_predictions[attributes])
RMSEs['Last3Average'] = calculate_RMSE(target[attributes], avg3[attributes])

In [54]:
# # Make ensemble predictions
# ensemble_all = DataFrame()
# for attribute in attributes:
#     ensemble_all[attribute] = (gbr_predictions[attribute] + bagr_predictions[attribute] + avg3[attribute]) / 3

# ensemble_gbr_bag = DataFrame()
# for attribute in attributes:
#     ensemble_gbr_bag[attribute] = (gbr_predictions[attribute] + bagr_predictions[attribute]) / 2

# ensemble_gbr_avg = DataFrame()
# for attribute in attributes:
#     ensemble_gbr_avg[attribute] = (gbr_predictions[attribute] + avg3[attribute]) / 2

# ensemble_bag_avg = DataFrame()
# for attribute in attributes:
#     ensemble_bag_avg[attribute] = (bagr_predictions[attribute] + avg3[attribute]) / 2

In [55]:
# RMSEs['Ensemble All'] = calculate_RMSE(target[attributes], ensemble_all)
# RMSEs['Ensemble GBR+Bag'] = calculate_RMSE(target[attributes], ensemble_gbr_bag)
# RMSEs['Ensemble GBR+Avg'] = calculate_RMSE(target[attributes], ensemble_gbr_avg)
# RMSEs['Ensemble Bag+Avg'] = calculate_RMSE(target[attributes], ensemble_bag_avg)

In [56]:
RMSEs

,Actual Variation,GBR,Bagging,Last3Average
PASS,17.091035,17.071217,17.171055,18.079626
PBLK,15.672840,16.077398,16.392842,16.078430
RECV,12.804767,13.067022,13.210863,13.343500
RUN,13.607940,13.919444,14.157834,14.634687
RBLK,13.066923,16.206617,16.710956,13.435114
RDEF,15.517219,15.956412,16.217593,16.697040
TACK,16.365188,17.370022,17.499051,17.612193
PRSH,12.931643,13.025733,13.205710,13.619227
COV,15.459220,15.593172,16.019597,16.838100


In [57]:
merged = bagr_predictions.merge(all_games[['GameNum','Season','Week','Date']],left_on='GameNum',right_on='GameNum')
merged.drop_duplicates().sort_values(by='GameNum')

,PASS,Team,GameNum,PBLK,RECV,RUN,RBLK,RDEF,TACK,PRSH,COV,Season,Week,Date
0,59.018750,Arizona Cardinals,185,70.510000,68.285000,62.150000,69.733333,61.830000,56.800000,64.943750,69.028571,2017,1,2019-09-10 00:00:00
2,69.090000,Detroit Lions,185,67.862500,68.425000,65.510000,67.775000,58.778571,69.514286,62.216667,60.370000,2017,1,2019-09-10 00:00:00
4,65.843750,Arizona Cardinals,186,64.045000,69.650000,59.033333,69.666667,66.045000,71.214286,61.137500,62.142857,2017,2,2019-09-17 00:00:00
6,56.937500,Indianapolis Colts,186,66.220000,69.937500,59.866667,69.006250,58.137500,62.500000,64.950000,61.985714,2017,2,2019-09-17 00:00:00
8,53.750000,Arizona Cardinals,187,61.900000,68.885000,57.622222,71.077778,62.315000,64.900000,62.725000,68.021429,2017,3,2019-09-25 00:00:00
10,65.194444,Dallas Cowboys,187,75.368750,68.155000,63.450000,76.405000,64.966667,57.875000,64.840000,66.700000,2017,3,2019-09-25 00:00:00
12,61.000000,Arizona Cardinals,188,64.790000,70.025000,62.866667,67.766667,61.630000,59.121429,65.712500,63.700000,2017,4,2019-10-01 00:00:00
14,61.305556,San Francisco 49Ers,188,66.625000,65.775000,64.135714,66.262500,65.592857,74.225000,64.085000,62.235714,2017,4,2019-10-01 00:00:00
18,69.750000,Philadelphia Eagles,189,66.080000,65.045000,62.200000,70.850000,68.164286,57.691667,67.800000,62.128571,2017,5,2019-10-08 00:00:00
16,62.543750,Arizona Cardinals,189,64.695000,70.175000,61.666667,69.438889,63.935000,71.585714,61.950000,65.892857,2017,5,2019-10-08 00:00:00


In [58]:
best_bag.to_csv('best_bag2.csv')
best_boost.to_csv('best_boost.csv')

In [59]:
best_bag

,Attribute,Team,max_samples,n_estimators
0,PASS,Arizona Cardinals,16,16
1,PASS,Atlanta Falcons,4,14
2,PASS,Baltimore Ravens,4,8
3,PASS,Buffalo Bills,12,14
4,PASS,Carolina Panthers,10,18
5,PASS,Chicago Bears,2,12
6,PASS,Cincinnati Bengals,8,16
7,PASS,Cleveland Browns,2,18
8,PASS,Dallas Cowboys,10,18
9,PASS,Denver Broncos,20,20
